In [ ]:
import numpy as np
import time
import pandas as pd
from scipy.stats import norm
import scipy.optimize as opt
from scipy.interpolate import interp1d

from math import *
import math
import seaborn as sns
import matplotlib.pyplot as plt

import cvxpy as cp
from scipy.optimize import fsolve
from scipy.optimize import leastsq
from scipy.optimize import brentq
from scipy.optimize import minimize

from scipy.stats import genextreme
from scipy.integrate import cumtrapz

from scipy.special import beta
from scipy.special import betainc
from scipy.optimize import least_squares
import os

In [ ]:
Index_Price = pd.read_csv('data/Index_Price.csv', encoding = 'ISO-8859-1')
Index_Price['Date'] = pd.to_datetime(Index_Price.Date)
Index_Price = Index_Price.sort_values(by='Date', ascending=True).reset_index(drop=True)
Index_Price['Date'] = pd.to_datetime(Index_Price.Date)

In [ ]:
def bs_implied_vol(S, X, T, r, q, option_price, is_call=True):
    
    low, high = 0.001, 1.0
    while high - low > 1e-6:
        mid = (low + high) / 2
        d1 = (np.log(S/X) + (r -q + mid**2/2)*T) / (mid * np.sqrt(T))
        d2 = d1 - mid * np.sqrt(T)
        if is_call:
            bs_price = S*exp(-q*T) * norm.cdf(d1) - X * np.exp(-r*T) * norm.cdf(d2)
        else:
            bs_price = X * np.exp(-r*T) * norm.cdf(-d2) - S *exp(-q*T)* norm.cdf(-d1)
        if bs_price < option_price:
            low = mid
        else:
            high = mid
    return mid

In [ ]:
def find_width(x_range2, f_temp2, x_m, m, height_ratio):
    target_height = height_ratio * m
    f_interp = interp1d(x_range2, f_temp2, kind='linear', bounds_error=False, fill_value=0)
    

    x_left_candidates = []
    x_right_candidates = []
    

    x_left = x_range2[x_range2 <= x_m]
    if len(x_left) > 0:

        for i in range(len(x_left)-1,-1,-1):
            if (f_temp2[i] - target_height) * (f_temp2[i-1] - target_height) < 0:
                try:
                    x_candidate = brentq(lambda x: f_interp(x) - target_height, x_left[i-1], x_left[i])
                    x_left_candidates.append(x_candidate)
                except ValueError:
                    pass
    

    x_right = x_range2[x_range2 >= x_m]
    if len(x_right) > 0:
        for i in range(len(x_right)-1):
            if (f_temp2[np.where(x_range2 == x_right[i])[0][0]] - target_height) * \
               (f_temp2[np.where(x_range2 == x_right[i+1])[0][0]] - target_height) < 0:
                try:
                    x_candidate = brentq(lambda x: f_interp(x) - target_height, x_right[i], x_right[i+1])
                    x_right_candidates.append(x_candidate)
                except ValueError:
                    pass
    

    x1 = min(x_left_candidates, key=lambda x: abs(x - x_m), default=x_m)
    x2 = min(x_right_candidates, key=lambda x: abs(x - x_m), default=x_m)
    

    if np.isclose(x1, x_m) and not np.isclose(x2, x_m):
        w = 2 * (x2 - x_m)  
    elif np.isclose(x2, x_m) and not np.isclose(x1, x_m):
        w = 2 * (x_m - x1)   
    else:
        w = x2 - x1          
    
    return w, x1, x2

In [ ]:
def estimate_bandwidth(strikes, put_prices, S, r, T, q):

   
    atm_idx = np.argmin(np.abs(strikes - S))
    sigma = bs_implied_vol(S, strikes[atm_idx], T, r, q, put_prices[atm_idx], is_call=False)

    std_dev = S*np.exp(r*T) * np.sqrt(np.exp(sigma**2 * T) - 1)
    h_initial = 0.5 * std_dev
    
    h_temp = h_initial
    a_temp, _, _ = solve_pca(strikes, put_prices, S, r, q, T, h_temp)
    
    dz = 0.5 * h_temp
    z_grid = np.arange(strikes[0], strikes[-1] , dz)

    x_range = np.linspace(strikes[0], strikes[-1], len(z_grid))
    f_temp = np.sum([a_temp[k] * norm.pdf((x_range - z_grid[k])/h_temp)/h_temp for k in range(len(a_temp))], axis=0)
    
    x_range2 = x_range
    f_temp2 = f_temp

    m = np.max(f_temp2)                 
    x_m = x_range2[np.argmax(f_temp2)]     

    m_1 = 0.75 * m 
    w1, x1, x2 = find_width(x_range2, f_temp2, x_m, m, height_ratio=0.75)


    m_2 = 0.5 * m 
    w2, x11, x22 = find_width(x_range2, f_temp2, x_m, m, height_ratio=0.5)

    
    a75 = 0.7585
    a50 = 1.1774
    
    h0 = min(w1/a75,w2/a50)
    
    return 0.95 * h0  

In [ ]:
def solve_pca(strikes, put_prices, S, r, q, T, h):
  
    dz = 0.5 * h
    j = math.ceil((strikes[-1]-strikes[0])/dz) 
    z_grid = np.linspace((int(math.floor(strikes[0] / 100.0)) * 100), (int(math.ceil(strikes[-1] / 100.0)) * 100), num=j)

    K = len(z_grid)
        
    
    def kernel_integral(x, z, h):
        d = (x - z) / h
        return h * (norm.pdf(d) + d * norm.cdf(d))
    
    Phi = np.array([[kernel_integral(x, z, h) for z in z_grid] for x in strikes])

    a = cp.Variable(K, nonneg=True)
    
    objective = cp.Minimize(cp.sum_squares(put_prices*np.exp(r*T) - Phi @ a)/len(put_prices))
    constraints = [
        cp.sum(a) == 1
    ]
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.ECOS)
    
    return a.value, h, prob.value

In [ ]:
def find_quantile(target_probability, tol=1e-6):
    def f(x):
        cdff = np.sum(a_hat * norm.cdf((np.tile(x, [len(a_hat),1]).transpose()-z_grid)/h_optimal),axis=1)
        return (cdff-target_probability)

    low_x = 0
    high_x = strikes[-1]*2
    while high_x - low_x > tol:
        mid_x = (low_x + high_x) / 2
        if f(low_x) * f(mid_x) < 0:
            high_x = mid_x
        else:
            low_x = mid_x

    return (low_x + high_x) / 2

def equations_left(params, *args):
    c, loc, scale = params
    x, target_f, alpha = args  
    gev = genextreme(c, loc=loc, scale=scale)
    
    eq1 = gev.pdf(-x[0]) - target_f[0]
    eq2 = gev.pdf(-x[1]) - target_f[1]
    eq3 = gev.cdf(-x[1]) - (1 - alpha)
    return [eq1, eq2, eq3]

def equations_right(params, *args):
    c, loc, scale = params
    x, target_f, alpha = args  
    gev = genextreme(c, loc=loc, scale=scale)
    
    eq1 = gev.pdf(x[0]) - target_f[0]
    eq2 = gev.pdf(x[1]) - target_f[1]
    eq3 = gev.cdf(x[1]) - alpha
    return [eq1, eq2, eq3]
   
def find_gev_x(target_probability, mu, sigma, eta):
    x = genextreme.ppf(target_probability, eta, loc=mu, scale=sigma)
    return x

In [ ]:
Code = "XLK"
df11 = pd.read_csv('data/XLK_option_demo.csv', encoding = 'ISO-8859-1')


df11['date'] = pd.to_datetime(df11.date)
df66 = df11
df66['date'] = pd.to_datetime(df66.date)
dates = df66.date.value_counts().sort_index().index

In [ ]:
mm = 1000
num = len(dates)
inverse_cdf_estimate  = np.empty([mm,num])
inverse_cdf_estimate3 = np.empty([mm,num])
QF = [None]*num   
for jj in range(num):
    dff = df66[df66['date']==dates[jj]].reset_index(drop=True)    
    
    idx = np.where(Index_Price['Date'] == dates[jj])[0]
    S_T = Index_Price.iloc[idx+21,:][Code].item()  

    strikes = dff['K'].values
    put_prices = dff['put_price'].values
    S = np.mean(dff['S'].values)
    q = np.mean(dff['q'].values)
    r = np.mean(dff['r'].values)
    T = np.mean(dff['T'].values)/360


    h_optimal = estimate_bandwidth(strikes,put_prices, S, r, T, q)
    a_hat, _ ,summ= solve_pca(strikes, put_prices, S, r, q, T, h_optimal)


    dz = 0.5 * h_optimal
    j = math.ceil((strikes[-1]-strikes[0])/dz)
    z_grid = np.linspace((int(math.floor(strikes[0] / 100.0)) * 100), (int(math.ceil(strikes[-1] / 100.0)) * 100), num=j)


    x_range = np.linspace(strikes[0], strikes[-1], 1000)

    rnd = np.sum(a_hat * norm.pdf(np.tile(x_range, [len(a_hat),1]).transpose(), z_grid, h_optimal), axis=1)
    cdf = np.sum(a_hat * norm.cdf((np.tile(x_range, [len(a_hat),1]).transpose()-z_grid)/h_optimal),axis=1)   

    QF[jj] = np.interp(S_T, x_range, cdf)
    s = np.linspace(strikes[0], strikes[-1],1000 )


    a = 0
    X1 = find_quantile(cdf[a], tol=1e-6)
    X2 = find_quantile(cdf[a]+0.005, tol=1e-6)

    f1 = np.interp(X1, x_range, rnd)
    f2 = np.interp(X2, x_range, rnd)

    XX = [X1, X2] 
    target_f = [f1, f2]  
    a_L = cdf[a]+0.005
    S_L = find_quantile(a_L, tol=1e-6)

    initial_guess = [-0.21,-np.mean(strikes), np.std(strikes)]

    bounds = ([-0.5, -np.inf, 1e-6], [0.5, np.inf, np.inf])  
    result_L = least_squares(lambda p: equations_left(p, XX, target_f, a_L), 
                          initial_guess, 
                          bounds=bounds,
                          max_nfev=1000)
    etal, mul, sigmal = result_L.x

    

    b = -1
    XX1 = find_quantile(cdf[b], tol=1e-6)
    XX2 = find_quantile(cdf[b]-0.005, tol=1e-6)

    ff1 = np.interp(XX1, x_range, rnd)
    ff2 = np.interp(XX2, x_range, rnd)

    XX_R = [XX1, XX2]  
    target_f_R = [ff1, ff2]  
    a_R = cdf[b]-0.005
    S_R = find_quantile(a_R, tol=1e-6)

    initial_guess = [-0.21, np.mean(strikes), np.std(strikes)]

    bounds = ([-0.5, -np.inf, 1e-6], [0.5, np.inf, np.inf])  
    result_R = least_squares(lambda p: equations_right(p, XX_R, target_f_R, a_R), 
                          initial_guess, 
                          bounds=bounds,
                          max_nfev=1000)
    etar, mur, sigmar = result_R.x

    def find_quantile(target_probability, tol=1e-6):
        def f(x_range):
            cddff = np.sum(a_hat * norm.cdf((np.tile(x_range, [len(a_hat),1]).transpose()-z_grid)/h_optimal),axis=1)
            return (cddff-target_probability)

        low_x = 0
        high_x = strikes[-1]*2
        while high_x - low_x > tol:
            mid_x = (low_x + high_x) / 2
            if f(low_x) * f(mid_x) < 0:
                high_x = mid_x
            else:
                low_x = mid_x

        return (low_x + high_x) / 2


    for i in range(0,mm):
        target_probability = 0.0005+i*0.001
        if target_probability < a_L:
            inverse_cdf_estimate[i,jj] = -find_gev_x(1-target_probability,mul, sigmal, etal)
        elif ((target_probability >= a_L) & (target_probability <= a_R)):
            inverse_cdf_estimate[i,jj] = find_quantile(target_probability, tol=1e-6)
        else:
            inverse_cdf_estimate[i,jj] = find_gev_x(target_probability,mur, sigmar, etar)
        
        col = inverse_cdf_estimate[:, jj]
        positive_vals = col[col > 0]            
        if positive_vals.size > 0:
            first_positive = positive_vals[0] 
            col[col < 0] = first_positive       
            inverse_cdf_estimate[:, jj] = col   


In [ ]:
jz2 = pd.DataFrame(inverse_cdf_estimate) #The risk-neutral c.d.f of single-asset price S_T

In [ ]:
jz3 = pd.concat([pd.DataFrame(dates),pd.DataFrame(QF)],axis=1) #F^Q(S_{\tau}) 